<a href="https://colab.research.google.com/github/Jozdien/reward-side-channels/blob/main/Reward_Side_Channels_with_Checkpoints_SB3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Installations

In [ ]:
!pip install "stable-baselines3[extra]>=2.0.0a4"
!apt-get update && apt-get install swig cmake ffmpeg freeglut3-dev xvfb
!pip install gym[box2d]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:2 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
free

##Env & Training

In [ ]:
import datetime
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.callbacks import CheckpointCallback
# Save a checkpoint every 1000 steps
env = gym.make('LunarLander-v2', render_mode="rgb_array")
checkpoint_callback = CheckpointCallback(save_freq=1000, save_path='/content/drive/MyDrive/Reward Side Channels - Model Data/logs_current/',
                                         name_prefix='ppo_agent')

model = PPO(MlpPolicy, env, verbose = 1)
model.learn(total_timesteps=10000, callback=checkpoint_callback, reset_num_timesteps=False)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 88.3     |
|    ep_rew_mean     | -183     |
| time/              |          |
|    fps             | 916      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 90          |
|    ep_rew_mean          | -177        |
| time/                   |             |
|    fps                  | 709         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008809397 |
|    clip_fraction        | 0.0544      |
|    clip_range           | 0.2         |
|    entropy_loss   

In [ ]:
#model.learn(total_timesteps=3000, callback=checkpoint_callback, reset_num_timesteps=False) #running again in case of interrupted runtime

Saving a copy of logs in drive (longterm)

In [ ]:
import shutil

original = r'/content/drive/MyDrive/Reward Side Channels - Model Data/logs_current'
target = r'/content/drive/MyDrive/Reward Side Channels - Model Data/logs/ppo-agent-results-ep-20-'+ datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

shutil.copytree(original, target)

'/content/drive/MyDrive/Reward Side Channels - Model Data/logs/ppo-agent-results-ep-20-2023-05-15_20-08-08'

##Lunar Lander Display

In [ ]:
# Set up display; otherwise rendering will fail
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [ ]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay


def show_videos(video_path="", prefix=""):
    """
    Taken from https://github.com/eleurent/highway-env

    :param video_path: (str) Path to the folder containing videos
    :param prefix: (str) Filter the video, showing only the only starting with this prefix
    """
    html = []
    for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append(
            """<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>""".format(
                mp4, video_b64.decode("ascii")
            )
        )
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

In [ ]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv


def record_video(env_id, model, video_length=500, prefix="", video_folder="videos/"):
    """
    :param env_id: (str)
    :param model: (RL model)
    :param video_length: (int)
    :param prefix: (str)
    :param video_folder: (str)
    """
    eval_env = DummyVecEnv([lambda: gym.make("LunarLander-v2", render_mode="rgb_array")])
    # Start the video at step=0 and record 500 steps
    eval_env = VecVideoRecorder(
        eval_env,
        video_folder=video_folder,
        record_video_trigger=lambda step: step == 0,
        video_length=video_length,
        name_prefix=prefix,
    )

    obs = eval_env.reset()
    for _ in range(video_length):
        action, _ = model.predict(obs)
        obs, _, _, _ = eval_env.step(action)

    # Close the video recorder
    eval_env.close()

In [ ]:
record_video("LunarLander-v2", model, video_length=500, prefix="ppo-lunarlander")

Saving video to /content/videos/ppo-lunarlander-step-0-to-step-500.mp4
Moviepy - Building video /content/videos/ppo-lunarlander-step-0-to-step-500.mp4.
Moviepy - Writing video /content/videos/ppo-lunarlander-step-0-to-step-500.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/ppo-lunarlander-step-0-to-step-500.mp4


In [ ]:
show_videos("videos", prefix="ppo")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
